<a href="https://colab.research.google.com/github/andresbadillo/ConversionMatrizHoraria/blob/master/archivos_inhemeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cargar archivos de lectura de los medidores Inhemeter

In [ ]:
import pandas as pd
from google.colab import files

# Paso 1: Cargar el archivo de texto
uploaded = files.upload()
input_file_path = list(uploaded.keys())[0]
output_file_path = input_file_path.replace('.txt', '.csv')

# Leer el archivo
with open(input_file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Eliminar las primeras 6 filas
lines = lines[6:]

# Identificar la línea donde comienzan los datos
data_start_index = None
for i, line in enumerate(lines):
    if line.startswith("No.|Date Time"):
        data_start_index = i
        break

# Extraer los datos y la cabecera
header = lines[data_start_index].strip().split('|')
data_lines = lines[data_start_index + 1:]

# Crear una lista de diccionarios para almacenar los datos
data = []
for line in data_lines:
    values = line.strip().split('|')
    if len(values) == len(header):
        data.append(dict(zip(header, values)))

# Crear un DataFrame
df = pd.DataFrame(data)

# Convertir las columnas numéricas y de fecha al tipo correcto
for col in df.columns:
    try:
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        try:
            df[col] = pd.to_datetime(df[col])
        except ValueError:
            pass

# Eliminar filas donde "Date Time" tenga datos de 30 minutos
df = df[~df['Date Time'].dt.minute.isin([30])]

# Eliminar la columna "No."
df = df.drop(columns=['No.'])

# Calcular la energía consumida por hora para cada energía
df['kWhR'] = df['Positive active energy (A+) total(kWh)'].diff().shift(-1)
df['kWhD'] = df['Negative active energy (A-) total(kWh)'].diff().shift(-1)
df['kVarR'] = df['Positive reactive energy (Q+) total(kvarh)'].diff().shift(-1)
df['kVarD'] = df['Negative reactive energy (Q-) total(kvarh)'].diff().shift(-1)

# Exportar a CSV con delimitador ";"
df.to_csv(output_file_path, index=False, encoding='utf-8', sep=';')

print(f"El archivo CSV ha sido creado: {output_file_path}")


## Exportar la matriz horaria

In [ ]:
# Eliminar columnas innecesarias
df = df.drop(columns=[
    'Positive active energy (A+) total(kWh)',
    'Negative active energy (A-) total(kWh)',
    'Positive reactive energy (Q+) total(kvarh)',
    'Negative reactive energy (Q-) total(kvarh)'
])

# Separar la columna "Date Time" en "Date" y "Time"
df['Date'] = df['Date Time'].dt.date
df['Time'] = df['Date Time'].dt.hour
df = df.drop(columns=['Date Time'])

# Reordenar columnas para que "Date" y "Time" estén al principio
columns_order = ['Date', 'Time'] + [col for col in df.columns if col not in ['Date', 'Time']]
df = df[columns_order]

# Exportar provisionalmente a un archivo CSV
provisional_output_path = 'provisional.csv'
df.to_csv(provisional_output_path, index=False, encoding='utf-8', sep=';')

print(f"El archivo provisional CSV ha sido creado: {provisional_output_path}")

# Crear una matriz horaria con una columna independiente para cada tipo de energía
df_melted = df.melt(id_vars=['Date', 'Time'], value_vars=['kWhR', 'kWhD', 'kVarR', 'kVarD'],
                    var_name='Energy Type', value_name='Value')

# Pivotar la columna "Time" para obtener una matriz horaria
df_pivot = df_melted.pivot_table(
    index=['Date', 'Energy Type'],
    columns='Time',
    values='Value',
    aggfunc='sum'
)

# Aplanar el índice de columnas para un formato tipo matríz
df_pivot.columns = [f"Hour{col}" for col in df_pivot.columns]
df_pivot.reset_index(inplace=True)

# Exportar la matriz horaria a un archivo CSV
matrix_output_path = 'hourly_matrix.csv'
df_pivot.to_csv(matrix_output_path, index=False, encoding='utf-8', sep=';')

print(f"El archivo matriz horaria CSV ha sido creado: {matrix_output_path}")

## Transformar otro tipo de archivo en matriz horaria

In [ ]:
import pandas as pd
from google.colab import files

# Paso 1: Cargar el archivo de Excel
uploaded = files.upload()
input_file_path = list(uploaded.keys())[0]
output_file_path = input_file_path.replace('.xlsx', '_processed.csv')

# Leer el archivo Excel
df = pd.read_excel(input_file_path)

# Eliminar la columna No.
df = df.drop(columns=['No.'], errors='ignore')

# Renombrar las columnas
df = df.rename(columns={
    'Positive active energy (A+) total(kWh)': 'kWhR',
    'Negative active energy (A-) total(kWh)': 'kWhD',
    'Positive reactive energy (Q+) total(kvarh)': 'kVarR',
    'Negative reactive energy (Q-) total(kvarh)': 'kVarD'
})

# Convertir la columna de fecha y hora al formato datetime
df['Date Time'] = pd.to_datetime(df['Date Time'])

# Extraer la fecha, la hora y los minutos
df['Date'] = df['Date Time'].dt.date
df['Hour'] = df['Date Time'].dt.hour
df['Minute'] = df['Date Time'].dt.minute

# Ajustar la hora para la agrupación correcta
df['Adjusted Hour'] = df['Hour']
df.loc[df['Minute'] == 0, 'Adjusted Hour'] = df['Hour'] - 1

df_grouped = df[df['Minute'].isin([15, 30, 45]) | ((df['Minute'] == 0) & (df['Adjusted Hour'] >= 0))]
df_grouped = df_grouped.drop(columns=['Date Time', 'Hour', 'Minute']).groupby(['Date', 'Adjusted Hour']).sum().reset_index()
df_grouped = df_grouped.rename(columns={'Adjusted Hour': 'Hour'})

# Reordenar columnas
columns_order = ['Date', 'Hour'] + [col for col in df_grouped.columns if col not in ['Date', 'Hour', 'Date Time']]
df_grouped = df_grouped[columns_order]

# Crear una matriz horaria con una columna independiente para cada tipo de energía
df_melted = df_grouped.melt(id_vars=['Date', 'Hour'], value_vars=['kWhR', 'kWhD', 'kVarR', 'kVarD'],
                            var_name='Energy Type', value_name='Value')

# Pivotar la columna "Hour" para obtener una matriz horaria
df_pivot = df_melted.pivot_table(
    index=['Date', 'Energy Type'],
    columns='Hour',
    values='Value',
    aggfunc='sum'
)

# Aplanar el índice de columnas para un formato tipo matriz
df_pivot.columns = [f"Hour{col}" for col in df_pivot.columns]
df_pivot.reset_index(inplace=True)

# Exportar la matriz horaria a un archivo CSV
matrix_output_path = input_file_path.replace('.xlsx', '_hourly_matrix.csv')
df_pivot.to_csv(matrix_output_path, index=False, encoding='utf-8', sep=';')

print(f"El archivo matriz horaria CSV ha sido creado: {matrix_output_path}")

# Transformar archivos ISKRA

### Archivos en formato .xls
Se puede cargar uno o más archivos. La descarga automática en la carpeta `descargas` solo funciona para los primeros diez archivos, por lo tanto se recomienda cargar hasta 10 archivos al tiempo

In [ ]:
import pandas as pd
from google.colab import files
import time

# Paso 1: Cargar varios archivos de Excel
uploaded = files.upload()  # Permite cargar múltiples archivos
output_paths = []  # Lista para guardar las rutas de los archivos de salida

# Iterar sobre cada archivo cargado
for input_file_path in uploaded.keys():
    output_file_path = input_file_path.replace('.xls', '_processed.csv')
    output_paths.append(output_file_path)

    # Leer el archivo Excel, seleccionando solo la pestaña "Sheet2"
    df = pd.read_excel(input_file_path, sheet_name='Sheet2')

    # Eliminar la primera columna "A"
    df = df.iloc[:, 1:]

    # Eliminar las primeras 6 filas
    df = df.iloc[5:].reset_index(drop=True)

    # Colocar la primera fila como encabezados
    df.columns = df.iloc[0]
    df = df.drop(0).reset_index(drop=True)

    # Eliminar columnas que no sean las especificadas
    columnas_permitidas = ['Hora', '1.9 [Wh]', '3.9 [varh]', '2.9 [Wh]', '4.9 [varh]']
    df = df[[col for col in df.columns if col in columnas_permitidas]]

    # Renombrar las columnas
    df.rename(columns={df.columns[0]: 'Date Time'}, inplace=True)
    df = df.rename(columns={
        '1.9 [Wh]': 'kWhR',
        '2.9 [Wh]': 'kWhD',
        '3.9 [varh]': 'kVarR',
        '4.9 [varh]': 'kVarD'
    })

    # Eliminar filas donde la columna "kWhR" tenga un valor NaN
    df = df.dropna(subset=['kWhR']).reset_index(drop=True)

    # Separar la columna "Date Time" en "Date" y "Time"
    df[['Date', 'Time']] = df['Date Time'].str.split(' ', expand=True, n=1)
    df = df.drop(columns=['Date Time'])
    df = df[['Date', 'Time'] + [col for col in df.columns if col not in ['Date', 'Time']]]

    # Convertir la columna "Date" a formato de fecha
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

    # Eliminar espacios adicionales y convertir la columna "Time" a formato 24 horas
    df['Time'] = df['Time'].str.strip()  # Eliminar espacios adicionales
    df['Time'] = df['Time'].str.replace('a.m.', 'AM').str.replace('p.m.', 'PM')
    df['Time'] = df['Time'].str.replace(r'^0:', '12:', regex=True)
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce').dt.strftime('%H:%M:%S')

    # Asegurarse de que la columna "Time" sea de tipo datetime
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.strftime('%H:%M')

    # Extraer la fecha, la hora y los minutos
    df['Date'] = df['Date'].dt.date
    df['Hour'] = df['Time'].str.split(':').str[0].astype(int)
    df['Minute'] = df['Time'].str.split(':').str[1].astype(int)

    # Ajustar la hora para la agrupación correcta
    df['Adjusted Hour'] = df['Hour']
    df.loc[df['Minute'] == 0, 'Adjusted Hour'] = df['Hour'] - 1

    # Filtrar solo los minutos correspondientes a 15, 30, 45 y 0 minutos de las horas válidas
    df_grouped = df[df['Minute'].isin([15, 30, 45]) | ((df['Minute'] == 0) & (df['Adjusted Hour'] >= 0))]

    # Agrupar por fecha y hora ajustada, luego sumar los valores
    df_grouped = df_grouped.drop(columns=['Time', 'Hour', 'Minute']).groupby(['Date', 'Adjusted Hour']).sum().reset_index()
    df_grouped = df_grouped.rename(columns={'Adjusted Hour': 'Hour'})

    # Validar columnas disponibles para la conversión
    disponibles = df_grouped.columns.tolist()
    columnas_energia = ['kWhR', 'kWhD', 'kVarR', 'kVarD']
    columnas_validas = [col for col in columnas_energia if col in disponibles]

    # Convertir los valores de energía de Wh a kWh
    df_grouped[columnas_validas] = df_grouped[columnas_validas] / 1000

    # Crear una matriz horaria con una columna independiente para cada tipo de energía
    df_melted = df_grouped.melt(id_vars=['Date', 'Hour'], value_vars=columnas_validas,
                                var_name='Energy Type', value_name='Value')

    # Convertir las horas a enteros para evitar fracciones y asegurarnos de que todas las horas de 0 a 23 sean representadas
    df_melted['Hour'] = df_melted['Hour'].astype(int)

    # Pivotar la columna "Hour" para obtener una matriz horaria
    df_pivot = df_melted.pivot_table(
        index=['Date', 'Energy Type'],
        columns='Hour',
        values='Value',
        aggfunc='sum'
    )

    # Aplanar el índice de columnas para un formato tipo matriz
    df_pivot.columns = [f"Hour{col}" for col in df_pivot.columns]
    df_pivot.reset_index(inplace=True)

    # Exportar la matriz horaria a un archivo CSV
    matrix_output_path = input_file_path.replace('.xls', '_hourly_matrix.csv')
    df_pivot.to_csv(matrix_output_path, index=False, encoding='utf-8', sep=';')
    files.download(matrix_output_path)

    print(f"El archivo matriz horaria CSV ha sido creado: {matrix_output_path}")
    time.sleep(1)

### Archivos en formato .csv

In [ ]:
import pandas as pd
from google.colab import files

# Paso 1: Cargar el archivo CSV
uploaded = files.upload()
input_file_path = list(uploaded.keys())[0]
output_file_path = input_file_path.replace('.csv', '_processed.csv')

# Leer el archivo CSV
df = pd.read_csv(input_file_path, delimiter=';', encoding='utf-8')

# Eliminar las primeras 3 filas
#df = df.iloc[3:].reset_index(drop=True)
#display(df)

# Colocar la primera fila como encabezados
#df.columns = df.iloc[0]
#df = df.drop(0).reset_index(drop=True)
#display(df)

# Eliminar columnas que no sean las especificadas
columnas_permitidas = ['Hora', '1.9', '3.9', '2.9', '4.9']
df = df[[col for col in df.columns if col in columnas_permitidas]]
#display(df)

# Renombrar las columnas
df = df.rename(columns={
    'Hora': 'Date Time',
    '1.9': 'kWhR',
    '2.9': 'kWhD',
    '3.9': 'kVarR',
    '4.9': 'kVarD'
})
#display(df)

# Eliminar filas donde la columna "Hora" tenga un valor NaN
#df = df.dropna(subset=['Hora']).reset_index(drop=True)
#display(df)

# Separar la columna "Date Time" en "Date" y "Time"
df[['Date', 'Time']] = df['Date Time'].str.split(' ', expand=True, n=1)
df = df.drop(columns=['Date Time'])
df = df[['Date', 'Time'] + [col for col in df.columns if col not in ['Date', 'Time']]]

# Convertir la columna "Date" a formato de fecha
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Eliminar espacios adicionales y convertir la columna "Time" a formato 24 horas
df['Time'] = df['Time'].str.strip()
df['Time'] = df['Time'].str.replace('a.m.', 'AM').str.replace('p.m.', 'PM')
df['Time'] = df['Time'].str.replace(r'^0:', '12:', regex=True)
df['Time'] = pd.to_datetime(df['Time'], errors='coerce').dt.strftime('%H:%M:%S')

df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.strftime('%H:%M')

# Extraer la fecha, la hora y los minutos
df['Date'] = df['Date'].dt.date
df['Hour'] = df['Time'].str.split(':').str[0].astype(int)
df['Minute'] = df['Time'].str.split(':').str[1].astype(int)

# Ajustar la hora para la agrupación correcta
df['Adjusted Hour'] = df['Hour']
df.loc[df['Minute'] == 0, 'Adjusted Hour'] = df['Hour'] - 1

# Filtrar solo los minutos correspondientes a 15, 30, 45 y 0 minutos de las horas válidas
df_grouped = df[df['Minute'].isin([15, 30, 45]) | ((df['Minute'] == 0) & (df['Adjusted Hour'] >= 0))]

# Agrupar por fecha y hora ajustada, luego sumar los valores
df_grouped = df_grouped.drop(columns=['Time', 'Hour', 'Minute']).groupby(['Date', 'Adjusted Hour']).sum().reset_index()
df_grouped = df_grouped.rename(columns={'Adjusted Hour': 'Hour'})

# Reordenar columnas
columns_order = ['Date', 'Hour'] + [col for col in df_grouped.columns if col not in ['Date', 'Hour', 'Date Time']]
df_grouped = df_grouped[columns_order]

# Validar columnas disponibles para la conversión
disponibles = df_grouped.columns.tolist()
columnas_energia = ['kWhR', 'kWhD', 'kVarR', 'kVarD']
columnas_validas = [col for col in columnas_energia if col in disponibles]

# Convertir los valores de energía de Wh a kWh
df_grouped[columnas_validas] = df_grouped[columnas_validas] / 1000

# Crear una matriz horaria con una columna independiente para cada tipo de energía
df_melted = df_grouped.melt(id_vars=['Date', 'Hour'], value_vars=columnas_validas,
                            var_name='Energy Type', value_name='Value')

df_melted['Hour'] = df_melted['Hour'].astype(int)

df_pivot = df_melted.pivot_table(
    index=['Date', 'Energy Type'],
    columns='Hour',
    values='Value',
    aggfunc='sum'
)

df_pivot.columns = [f"Hour{col}" for col in df_pivot.columns]
df_pivot.reset_index(inplace=True)

# Exportar la matriz horaria a un archivo CSV
matrix_output_path = input_file_path.replace('.csv', '_hourly_matrix.csv')
df_pivot.to_csv(matrix_output_path, index=False, encoding='utf-8', sep=';')

print(f"El archivo matriz horaria CSV ha sido creado: {matrix_output_path}")

### Borrar carpeta `content`

In [51]:
import os

# Ruta de la carpeta predeterminada en Google Colab
folder_path = '/content'

# Eliminar todos los archivos en la carpeta
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        os.remove(file_path)

print("\nTodos los archivos cargados han sido eliminados de la carpeta content de Google Colab.")


Todos los archivos cargados han sido eliminados de la carpeta content de Google Colab.
